In [12]:
# coefficient matching algorithm for constrained terms
# x = 1/2*q1 + 1/4*q2 - 1/2*q3 - 1/4*q4

import numpy as np
import random, math
import copy

A = np.array([[1, -1, 1], [-2, 3, 4], [1, -4, -1]])
b = np.array([1, -1, 2])

[Dimension, ny] = A.shape
qubits = 2

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,-2*(l+1))*pow(A[k][i],2)
            cef2 = pow(2,-(l+1)+1)*A[k][i]*b[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, -(l1+1)-(l2+1)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, -(l1+1)-(l2+1)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      


# Matrix Q is
[[-3.5    1.5    0.     0.    -5.5   -2.75   5.5    2.75  -4.    -2.
   4.     2.   ]
 [ 0.    -2.125  0.     0.    -2.75  -1.375  2.75   1.375 -2.    -1.
   2.     1.   ]
 [ 0.     0.     6.5    1.5    5.5    2.75  -5.5   -2.75   4.     2.
  -4.    -2.   ]
 [ 0.     0.     0.     2.875  2.75   1.375 -2.75  -1.375  2.     1.
  -2.    -1.   ]
 [ 0.     0.     0.     0.    18.5    6.5    0.     0.     7.5    3.75
  -7.5   -3.75 ]
 [ 0.     0.     0.     0.     0.     7.625  0.     0.     3.75   1.875
  -3.75  -1.875]
 [ 0.     0.     0.     0.     0.     0.    -5.5    6.5   -7.5   -3.75
   7.5    3.75 ]
 [ 0.     0.     0.     0.     0.     0.     0.    -4.375 -3.75  -1.875
   3.75   1.875]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     9.5    4.5
   0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     3.625
   0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  -0.5    4.5  ]
 [ 0.     0.     0.     0.

In [13]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-3.5, ('q2','q2'):-2.125, ('q3','q3'):6.5, ('q4','q4'):2.875, ('q5','q5'):18.5, ('q6','q6'):7.625, ('q7','q7'):-5.5, ('q8','q8'):-4.375, ('q9','q9'):9.5, ('q10','q10'):3.625, ('q11','q11'):-0.5, ('q12','q12'):-1.375}

quadratic = {('q1','q2'):1.5, ('q1','q5'):-5.5, ('q1','q6'):-2.75, ('q1','q7'):5.5, ('q1','q8'):2.75, ('q1','q9'):-4.0, ('q1','q10'):-2.0, ('q1','q11'):4.0, ('q1','q12'):2.0, ('q2','q5'):-2.75, ('q2','q6'):-1.375, ('q2','q7'):2.75, ('q2','q8'):1.375, ('q2','q9'):-2.0, ('q2','q10'):-1.0, ('q2','q11'):2.0, ('q2','q12'):1.0, ('q3','q4'):1.5, ('q3','q5'):5.5, ('q3','q6'):2.75, ('q3','q7'):-5.5, ('q3','q8'):-2.75, ('q3','q9'):4.0, ('q3','q10'):2.0, ('q3','q11'):-4.0, ('q3','q12'):-2.0, ('q4','q5'):2.75, ('q4','q6'):1.375, ('q4','q7'):-2.75, ('q4','q8'):-1.375, ('q4','q9'):2.0, ('q4','q10'):1.0, ('q4','q11'):-2.0, ('q4','q12'):-1.0, ('q5','q6'):6.5, ('q5','q9'):7.5, ('q5','q10'):3.75, ('q5','q11'):-7.5, ('q5','q12'):-3.75, ('q6','q9'):3.75, ('q6','q10'):1.875, ('q6','q11'):-3.75, ('q6','q12'):-1.875, ('q7','q8'):6.5, ('q7','q9'):-7.5, ('q7','q10'):-3.75, ('q7','q11'):7.5, ('q7','q12'):3.75, ('q8','q9'):-3.75, ('q8','q10'):-1.875, ('q8','q11'):3.75, ('q8','q12'):1.875, ('q9','q10'):4.5, ('q11','q12'):4.5}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc. chain_b.
0   0   1   0   0  1  0  0  0  0  1  0  0   -6.0      72      0.0
1   0   1   0   0  0  0  1  0  0  1  1  0 -5.625      40      0.0
2   0   1   0   0  0  0  0  0  0  1  0  0 -5.625      63      0.0
3   1   1   0   0  0  0  0  0  0  1  0  0 -5.625      18      0.0
4   1   1   0   0  1  0  0  0  0  0  1  0 -5.625      41      0.0
5   0   0   0   0  0  0  0  0  0  1  0  0   -5.5      32      0.0
6   1   0   0   0  0  0  0  0  0  1  0  1   -5.5       6      0.0
7   1   1   0   0  0  0  0  0  0  0  1  0 -5.375      23      0.0
8   0   1   0   0  0  0  0  0  0  1  1  0 -5.375      43      0.0
9   0   0   0   0  0  0  1  0  0  1  0  0 -5.375      39      0.0
10  1   0   0   0  1  0  0  0  0  1  0  1 -5.375       2      0.0
11  1   0   0   0  0  0  0  0  0  0  1  0 -5.125       4      0.0
12  0   0   0   0  0  1  0  0  0  1  1  0 -5.125      87      0.0
13  0   1   0   0  0  1  0  0  0  1  1  0 -5.125     110      0.0
14  0   0 

In [14]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc. chain_b.
0   0   1   0   0  1  0  0  0  0  1  0  0   -6.0      38      0.0
1   0   1   0   0  0  0  1  0  0  1  1  0 -5.625      14      0.0
2   0   1   0   0  0  0  0  0  0  1  0  0 -5.625      14      0.0
3   1   1   0   0  0  0  0  0  0  1  0  0 -5.625       2      0.0
4   1   1   0   0  1  0  0  0  0  0  1  0 -5.625      61      0.0
78  1   1   0   0  1  0  0  0  0  0  1  0 -5.625       1 0.083333
81  1   1   0   0  1  0  0  0  0  0  1  0 -5.625       1 0.083333
5   0   0   0   0  0  0  0  0  0  1  0  0   -5.5       3      0.0
6   1   0   0   0  0  0  0  0  0  1  0  1   -5.5      24      0.0
7   1   0   0   0  1  0  0  0  0  1  0  1 -5.375      28      0.0
8   0   0   0   0  0  0  1  0  0  1  0  0 -5.375       2      0.0
9   0   1   0   0  0  0  0  0  0  1  1  0 -5.375      25      0.0
10  1   1   0   0  0  0  0  0  0  0  1  0 -5.375       9      0.0
50  0   1   0   0  0  0  0  0  0  1  1  0 -5.375       1 0.083333
94  1   0 

In [15]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc. chain_b.
0   0   1   0   0  1  0  0  0  0  1  0  0   -6.0      68      0.0
24  0   1   0   0  1  0  0  0  0  1  0  0   -6.0       3 0.083333
1   1   1   0   0  0  0  0  0  0  1  0  0 -5.625     101      0.0
2   0   1   0   0  0  0  1  0  0  1  1  0 -5.625      77      0.0
3   0   1   0   0  0  0  0  0  0  1  0  0 -5.625     138      0.0
30  0   1   0   0  0  0  1  0  0  1  1  0 -5.625       2 0.083333
31  1   1   0   0  0  0  0  0  0  1  0  0 -5.625       1 0.083333
32  0   1   0   0  0  0  0  0  0  1  0  0 -5.625       7 0.083333
4   0   0   0   0  0  0  0  0  0  1  0  0   -5.5      55      0.0
5   1   0   0   0  0  0  0  0  0  1  0  1   -5.5      44      0.0
35  0   0   0   0  0  0  0  0  0  1  0  0   -5.5       1 0.083333
6   0   0   0   0  0  0  1  0  0  1  0  0 -5.375      53      0.0
7   1   0   0   0  1  0  0  0  0  1  0  1 -5.375      17      0.0
8   0   1   0   0  0  0  0  0  0  1  1  0 -5.375      29      0.0
39  0   0 

In [16]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc. chain_b.
0    0   1   0   0  1  0  0  0  0  1  0  0   -6.0      51      0.0
63   0   1   0   0  1  0  0  0  0  1  0  0   -6.0       1 0.083333
1    0   1   0   0  0  0  1  0  0  1  1  0 -5.625      69      0.0
2    0   1   0   0  0  0  0  0  0  1  0  0 -5.625      21      0.0
3    1   1   0   0  0  0  0  0  0  1  0  0 -5.625      13      0.0
4    1   1   0   0  1  0  0  0  0  0  1  0 -5.625      31      0.0
29   1   1   0   0  1  0  0  0  0  0  1  0 -5.625       6 0.083333
70   0   1   0   0  0  0  1  0  0  1  1  0 -5.625       1 0.083333
5    1   0   0   0  0  0  0  0  0  1  0  1   -5.5      28      0.0
6    0   0   0   0  0  0  0  0  0  1  0  0   -5.5      20      0.0
7    0   0   0   0  0  0  1  0  0  1  0  0 -5.375      57      0.0
8    0   1   0   0  0  0  0  0  0  1  1  0 -5.375       9      0.0
9    1   1   0   0  0  0  0  0  0  0  1  0 -5.375      32      0.0
10   1   0   0   0  1  0  0  0  0  1  0  1 -5.375      25     

In [17]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc. chain_b.
0   0   1   0   0  1  0  0  0  0  1  0  0   -6.0       6      0.0
1   1   1   0   0  1  0  0  0  0  0  1  0 -5.625      43      0.0
2   0   1   0   0  0  0  1  0  0  1  1  0 -5.625      32      0.0
3   1   1   0   0  0  0  0  0  0  1  0  0 -5.625       4      0.0
4   0   1   0   0  0  0  0  0  0  1  0  0 -5.625       6      0.0
95  1   1   0   0  1  0  0  0  0  0  1  0 -5.625       1 0.083333
5   0   0   0   0  0  0  0  0  0  1  0  0   -5.5      12      0.0
6   1   0   0   0  0  0  0  0  0  1  0  1   -5.5      16      0.0
7   0   1   0   0  0  0  0  0  0  1  1  0 -5.375       8      0.0
8   1   1   0   0  0  0  0  0  0  0  1  0 -5.375      29      0.0
9   1   0   0   0  1  0  0  0  0  1  0  1 -5.375      36      0.0
10  0   0   0   0  0  0  1  0  0  1  0  0 -5.375      13      0.0
11  0   0   0   0  0  1  0  0  0  1  1  0 -5.125       9      0.0
12  0   0   0   0  1  0  0  0  0  1  1  1 -5.125     116      0.0
13  0   0 